In [1]:
import os
from argparse import Namespace
from collections import Counter
import json
import re
import string

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm



In [2]:
class Vocabulary:
    def __init__(self, token_to_idx=None):
        self._token_to_idx = token_to_idx if token_to_idx else {}
        self._idx_to_token = {idx: token for token, idx in self._token_to_idx.items()}

    def to_serializable(self):
        return {'token_to_idx': self._token_to_idx}

    @classmethod
    def from_serializable(cls, contents):
        return cls(**contents)

    def add_token(self, token):
        if token in self._token_to_idx:
            return self._token_to_idx[token]
        index = len(self._token_to_idx)
        self._token_to_idx[token] = index
        self._idx_to_token[index] = token
        return index

    def add_many(self, tokens):
        return [self.add_token(token) for token in tokens]

    def lookup_token(self, token):
        return self._token_to_idx[token]

    def lookup_index(self, index):
        if index not in self._idx_to_token:
            raise KeyError(f"Index {index} not in vocabulary")
        return self._idx_to_token[index]

    def __str__(self):
        return f"<Vocabulary(size={len(self)})>"

    def __len__(self):
        return len(self._token_to_idx)



In [3]:
class SequenceVocabulary(Vocabulary):
    def __init__(self, token_to_idx=None, unk_token="<UNK>", mask_token="<MASK>",
                 begin_seq_token="<BEGIN>", end_seq_token="<END>"):
        super().__init__(token_to_idx)

        self._unk_token = unk_token
        self._mask_token = mask_token
        self._begin_seq_token = begin_seq_token
        self._end_seq_token = end_seq_token

        self.unk_index = self.add_token(self._unk_token)
        self.mask_index = self.add_token(self._mask_token)
        self.begin_seq_index = self.add_token(self._begin_seq_token)
        self.end_seq_index = self.add_token(self._end_seq_token)

    def to_serializable(self):
        contents = super().to_serializable()
        contents.update({
            'unk_token': self._unk_token,
            'mask_token': self._mask_token,
            'begin_seq_token': self._begin_seq_token,
            'end_seq_token': self._end_seq_token
        })
        return contents

    def lookup_token(self, token):
        return self._token_to_idx.get(token, self.unk_index)



In [4]:
class SurnameVectorizer:
    def __init__(self, char_vocab, nationality_vocab):
        self.char_vocab = char_vocab
        self.nationality_vocab = nationality_vocab

    def vectorize(self, surname, vector_length=-1):
        indices = [self.char_vocab.begin_seq_index]
        indices.extend(self.char_vocab.lookup_token(token) for token in surname)
        indices.append(self.char_vocab.end_seq_index)

        vector_length = len(indices) - 1 if vector_length < 0 else vector_length

        from_vector = np.full(vector_length, self.char_vocab.mask_index, dtype=np.int64)
        from_indices = indices[:-1]
        from_vector[:len(from_indices)] = from_indices

        to_vector = np.full(vector_length, self.char_vocab.mask_index, dtype=np.int64)
        to_indices = indices[1:]
        to_vector[:len(to_indices)] = to_indices

        return from_vector, to_vector

    @classmethod
    def from_dataframe(cls, surname_df):
        char_vocab = SequenceVocabulary()
        nationality_vocab = Vocabulary()

        for _, row in surname_df.iterrows():
            for char in row.surname:
                char_vocab.add_token(char)
            nationality_vocab.add_token(row.nationality)

        return cls(char_vocab, nationality_vocab)

    @classmethod
    def from_serializable(cls, contents):
        char_vocab = SequenceVocabulary.from_serializable(contents['char_vocab'])
        nat_vocab = Vocabulary.from_serializable(contents['nationality_vocab'])
        return cls(char_vocab, nat_vocab)

    def to_serializable(self):
        return {
            'char_vocab': self.char_vocab.to_serializable(),
            'nationality_vocab': self.nationality_vocab.to_serializable()
        }



In [5]:
class SurnameDataset(Dataset):
    def __init__(self, surname_df, vectorizer):
        self.surname_df = surname_df
        self._vectorizer = vectorizer
        self._max_seq_length = max(len(s) for s in surname_df.surname) + 2  # +2 for begin/end tokens

        self._split_df = {
            'train': surname_df[surname_df.split == 'train'],
            'val': surname_df[surname_df.split == 'val'],
            'test': surname_df[surname_df.split == 'test']
        }
        self.set_split('train')

    @classmethod
    def load_dataset_and_make_vectorizer(cls, csv_path):
        df = pd.read_csv(csv_path)
        return cls(df, SurnameVectorizer.from_dataframe(df))

    @classmethod
    def load_dataset_and_load_vectorizer(cls, csv_path, vectorizer_path):
        df = pd.read_csv(csv_path)
        vectorizer = cls.load_vectorizer_only(vectorizer_path)
        return cls(df, vectorizer)

    @staticmethod
    def load_vectorizer_only(vectorizer_path):
        with open(vectorizer_path) as fp:
            return SurnameVectorizer.from_serializable(json.load(fp))

    def save_vectorizer(self, vectorizer_path):
        with open(vectorizer_path, 'w') as fp:
            json.dump(self._vectorizer.to_serializable(), fp)

    def get_vectorizer(self):
        return self._vectorizer

    def set_split(self, split='train'):
        self._target_split = split
        self._target_df = self._split_df[split]
        self._target_size = len(self._target_df)

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        row = self._target_df.iloc[index]
        from_vec, to_vec = self._vectorizer.vectorize(row.surname, self._max_seq_length)
        nat_index = self._vectorizer.nationality_vocab.lookup_token(row.nationality)
        return {
            'x_data': from_vec,
            'y_target': to_vec,
            'class_index': nat_index
        }

    def get_num_batches(self, batch_size):
        return len(self) // batch_size



In [6]:
def generate_batches(dataset, batch_size, shuffle=True, drop_last=True, device="cpu"):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)
    for batch in dataloader:
        yield {k: v.to(device) for k, v in batch.items()}


def normalize_sizes(y_pred, y_true):
    if y_pred.dim() == 3:
        y_pred = y_pred.view(-1, y_pred.size(2))
    if y_true.dim() == 2:
        y_true = y_true.view(-1)
    return y_pred, y_true


def compute_accuracy(y_pred, y_true, mask_index):
    y_pred, y_true = normalize_sizes(y_pred, y_true)
    _, y_pred_indices = y_pred.max(dim=1)
    correct = (y_pred_indices == y_true).float()
    valid = (y_true != mask_index).float()
    return (correct * valid).sum().item() / valid.sum().item() * 100


def sequence_loss(y_pred, y_true, mask_index):
    y_pred, y_true = normalize_sizes(y_pred, y_true)
    return F.cross_entropy(y_pred, y_true, ignore_index=mask_index)


def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)


def handle_dirs(dirpath):
    os.makedirs(dirpath, exist_ok=True)



In [7]:
class SurnameGenerationModel(nn.Module):
    def __init__(self, char_embedding_size, char_vocab_size, rnn_hidden_size,
                 batch_first=True, padding_idx=0, dropout_p=0.5):
        super().__init__()
        self.emb = nn.Embedding(char_vocab_size, char_embedding_size, padding_idx=padding_idx)
        self.rnn = nn.GRU(char_embedding_size, rnn_hidden_size, batch_first=batch_first)
        self.fc = nn.Linear(rnn_hidden_size, char_vocab_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, x_in, apply_softmax=False):
        x_emb = self.emb(x_in)
        rnn_out, _ = self.rnn(x_emb)
        batch_size, seq_len, feat_size = rnn_out.shape

        # Flatten for linear layer
        rnn_out = rnn_out.contiguous().view(batch_size * seq_len, feat_size)
        output = self.fc(self.dropout(rnn_out))

        if apply_softmax:
            output = F.softmax(output, dim=1)

        return output.view(batch_size, seq_len, -1)



In [8]:
def sample_from_model(model, vectorizer, num_samples=1, sample_size=20, temperature=1.0):
    begin_idx = [[vectorizer.char_vocab.begin_seq_index]] * num_samples
    begin_tensor = torch.tensor(begin_idx, dtype=torch.int64)
    sequences = [begin_tensor]
    h_t = None

    for _ in range(sample_size):
        x_t = sequences[-1]
        emb_t = model.emb(x_t)
        rnn_out, h_t = model.rnn(emb_t, h_t)
        logits = model.fc(rnn_out.squeeze(1))
        probs = F.softmax(logits / temperature, dim=1)
        next_tokens = torch.multinomial(probs, num_samples=1)
        sequences.append(next_tokens)

    return torch.cat(sequences, dim=1)


def decode_samples(sampled_indices, vectorizer):
    vocab = vectorizer.char_vocab
    decoded = []
    for sample in sampled_indices:
        chars = []
        for idx in sample[1:]:  # Skip begin token
            if idx.item() == vocab.end_seq_index:
                break
            chars.append(vocab.lookup_index(idx.item()))
        decoded.append(''.join(chars))
    return decoded



In [9]:
def make_train_state(args):
    return {
        'stop_early': False,
        'early_stopping_step': 0,
        'early_stopping_best_val': float('inf'),
        'epoch_index': 0,
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': [],
        'test_loss': -1,
        'test_acc': -1,
        'model_filename': args.model_state_file
    }


def update_train_state(args, model, train_state):
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
    else:
        current_loss = train_state['val_loss'][-1]
        if current_loss < train_state['early_stopping_best_val']:
            torch.save(model.state_dict(), train_state['model_filename'])
            train_state['early_stopping_best_val'] = current_loss
            train_state['early_stopping_step'] = 0
        else:
            train_state['early_stopping_step'] += 1
            if train_state['early_stopping_step'] >= args.early_stopping_criteria:
                train_state['stop_early'] = True
    return train_state



In [10]:
args = Namespace(
    surname_csv="surnames_with_splits.csv",
    vectorizer_file="vectorizer.json",
    model_state_file="model.pth",
    save_dir="model_storage/char_rnn",
    char_embedding_size=64,
    rnn_hidden_size=64,
    batch_size=128,
    num_epochs=100,
    early_stopping_criteria=5,
    learning_rate=1e-3,
    seed=42,
    cuda=True,
    expand_filepaths_to_save_dir=True,
    reload_from_files=False
)

# 路径处理
if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = os.path.join(args.save_dir, args.vectorizer_file)
    args.model_state_file = os.path.join(args.save_dir, args.model_state_file)

# 设备设置
device = torch.device("cuda" if args.cuda and torch.cuda.is_available() else "cpu")


In [11]:
set_seed_everywhere(args.seed, args.cuda)
handle_dirs(args.save_dir)

# 数据加载
if args.reload_from_files:
    dataset = SurnameDataset.load_dataset_and_load_vectorizer(args.surname_csv, args.vectorizer_file)
else:
    dataset = SurnameDataset.load_dataset_and_make_vectorizer(args.surname_csv)
    dataset.save_vectorizer(args.vectorizer_file)

vectorizer = dataset.get_vectorizer()
mask_index = vectorizer.char_vocab.mask_index

# 模型初始化
model = SurnameGenerationModel(
    char_embedding_size=args.char_embedding_size,
    char_vocab_size=len(vectorizer.char_vocab),
    rnn_hidden_size=args.rnn_hidden_size,
    padding_idx=mask_index
).to(device)

optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
train_state = make_train_state(args)


In [12]:
try:
    for epoch in range(args.num_epochs):
        train_state['epoch_index'] = epoch

        # Training
        model.train()
        dataset.set_split('train')
        batch_generator = generate_batches(dataset, args.batch_size, device=device)
        progress_bar = tqdm(batch_generator,
                            total=dataset.get_num_batches(args.batch_size),
                            desc=f"Epoch {epoch} Train")

        running_loss = 0.0
        running_acc = 0.0
        for batch_idx, batch in enumerate(progress_bar):
            optimizer.zero_grad()

            y_pred = model(batch['x_data'])
            loss = sequence_loss(y_pred, batch['y_target'], mask_index)
            loss.backward()
            optimizer.step()

            acc = compute_accuracy(y_pred, batch['y_target'], mask_index)
            running_loss = (running_loss * batch_idx + loss.item()) / (batch_idx + 1)
            running_acc = (running_acc * batch_idx + acc) / (batch_idx + 1)

            progress_bar.set_postfix({
                'loss': f"{running_loss:.4f}",
                'acc': f"{running_acc:.2f}%"
            })

        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)

        # Validation
        model.eval()
        dataset.set_split('val')
        batch_generator = generate_batches(dataset, args.batch_size, device=device)
        progress_bar = tqdm(batch_generator,
                            total=dataset.get_num_batches(args.batch_size),
                            desc=f"Epoch {epoch} Val")

        val_loss = 0.0
        val_acc = 0.0
        with torch.no_grad():
            for batch_idx, batch in enumerate(progress_bar):
                y_pred = model(batch['x_data'])
                loss = sequence_loss(y_pred, batch['y_target'], mask_index)
                acc = compute_accuracy(y_pred, batch['y_target'], mask_index)

                val_loss = (val_loss * batch_idx + loss.item()) / (batch_idx + 1)
                val_acc = (val_acc * batch_idx + acc) / (batch_idx + 1)

                progress_bar.set_postfix({
                    'val_loss': f"{val_loss:.4f}",
                    'val_acc': f"{val_acc:.2f}%"
                })

        train_state['val_loss'].append(val_loss)
        train_state['val_acc'].append(val_acc)

        # Update state
        train_state = update_train_state(args, model, train_state)
        scheduler.step(val_loss)

        # Early stopping
        if train_state['stop_early']:
            break

        # Sampling demonstration
        model.cpu()
        samples = decode_samples(
            sample_from_model(model, vectorizer, num_samples=2, temperature=0.6),
            vectorizer
        )
        print(f"\nSamples after epoch {epoch}:")
        print("\n".join(samples))
        model.to(device)

except KeyboardInterrupt:
    print("Training interrupted")


Epoch 0 Val: 100%|██████████| 12/12 [00:00<00:00, 57.97it/s, val_loss=2.9292, val_acc=19.82%]



Samples after epoch 0:
Jauno
Tagirero


Epoch 1 Val: 100%|██████████| 12/12 [00:00<00:00, 58.25it/s, val_loss=2.6661, val_acc=24.08%]



Samples after epoch 1:
Manchula
Bero


Epoch 2 Val: 100%|██████████| 12/12 [00:00<00:00, 56.87it/s, val_loss=2.5750, val_acc=24.87%]


...

Epoch 74 Val: 100%|██████████| 12/12 [00:00<00:00, 57.14it/s, val_loss=2.2427, val_acc=33.18%]



Samples after epoch 74:
Hanlel
Araman


Epoch 75 Val: 100%|██████████| 12/12 [00:00<00:00, 57.14it/s, val_loss=2.2412, val_acc=32.96%]



Samples after epoch 75:
Curuner
Terino


Epoch 76 Val: 100%|██████████| 12/12 [00:00<00:00, 52.94it/s, val_loss=2.2411, val_acc=33.18%]


In [13]:
model.cpu()
final_samples = decode_samples(
    sample_from_model(model, vectorizer, num_samples=10, temperature=0.6),
    vectorizer
)

print("\nFinal Generated Samples:")
print("-" * 30)
for i, name in enumerate(final_samples):
    print(f"{i + 1}. {name}")


Final Generated Samples:
------------------------------
1. Arahas
2. Badyan
3. Baba
4. Catser
5. Girmond
6. Ficher
7. Salid
8. Ancham
9. Manger
10. Goren
